[KerasTuner](https://www.tensorflow.org/tutorials/keras/keras_tuner) helps you find a best set of your neural network's hyperparameters.

So let's give it a go!

# Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split
from tqdm.auto import tqdm

import tensorflow as tf 
# import tensorflow_addons as tfa
# !pip install -q -U keras-tuner
import kerastuner as kt # keras tuner!

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('seaborn-colorblind')

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings('ignore')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Config

In [ ]:
SEED = 42
NFOLD = 10
OUTPUT_DIR = ''

In [ ]:
# Logging is always nice for your experiment:)
def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = init_logger()
logger.info('Start Logging...')

# Load data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')

features = [f'cont{i}' for i in range(1, 15)]
target_col = 'target'

X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']
X_test = test.drop('id', axis=1)

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
print(X_test.shape)
X_test.head()

# Tuning NN with kerastuner
We use a simple MLP and tune the hyperparameters!

## Scaling
To make sure similar range across features

In [ ]:
prep = StandardScaler()
df = pd.concat([X_train[features], X_test[features]])
df[features] = prep.fit_transform(df[features].values)
X_test[features] = df[features].iloc[len(X_train):]
X_train[features] = df[features].iloc[:len(X_train)]

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
print(X_test.shape)
X_test.head()

## MLP

In [ ]:
# my default NN hyperparameters
params = {
    'input_dim': len(features),
    'input_dropout': 0.0,
    'hidden_layers': 3,
    'hidden_units': 256,
    'hidden_activation': 'relu',
    'lr': 1e-03,
    'dropout': 0.2,
    'batch_size': 128,
    'epochs': 192
}
logger.info('default NN params:')
logger.info(params)

def tuning_model(hp, params=params):
    """
    model tuning with KerasTuner
    """
    
    inputs = tf.keras.layers.Input(shape=(params['input_dim'], ))
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.Dense(hp.Int('num_units_1', 128, 512, step=128), activation=params['hidden_activation'])(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(hp.Float('dropout_1', 0.0, 0.5, step=0.1, default=0.5))(x)

    for i in range(hp.Int('num_layers', 1, 3)):
        x = tf.keras.layers.Dense(hp.Int(f'num_units_{i+2}', 128, 512, step=128))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(hp.Float(f'dropout_{i+2}', 0.0, 0.5, step=0.1, default=0.5))(x)
        
    # output
    out = tf.keras.layers.Dense(1, activation='linear', name = 'out')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=out)
   
    # compile
    loss = tf.keras.losses.MeanSquaredError()
    opt = tf.keras.optimizers.Adam(lr=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log'))
    model.compile(loss=loss, optimizer=opt, metrics=['mse'])
    
    return model

In [ ]:
# create a dataset for NN based on a task
train, valid, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.3, random_state=SEED)

train_set = {'X': train[features].values, 'y': train_y.values}
valid_set = {'X': valid[features].values, 'y': valid_y.values}  

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - RandomSearch, Hyperband, BayesianOptimization, and Sklearn.

Here we use the BayesianOptimization tuner.

In [ ]:
# define a custom tuner to tune the batch size
class MyTuner(kt.tuners.BayesianOptimization):
  def run_trial(self, trial, *args, **kwargs):
    # You can add additional HyperParameters for preprocessing and custom training loops
    # via overriding `run_trial`
    kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 128, 8192, step=128)
#     kwargs['epochs'] = trial.hyperparameters.Int('epochs', 10, 30)
    super(MyTuner, self).run_trial(trial, *args, **kwargs)

# instantiate KerasTuner
model_ft = lambda hp: tuning_model(hp, params)
tuner = MyTuner(
    hypermodel=model_ft,
    objective=kt.Objective('val_loss', direction='min'),
    num_initial_points=4,
    max_trials=20,
    overwrite=True)

# perform tuning
tuner.search(train_set['X'], train_set['y'], verbose=2,
             epochs = 8, validation_data = (valid_set['X'], valid_set['y']))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)

# disp best params
logger.info('Best hyperparameters:')
logger.info(best_hps.values)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

# Fit with KFold
Let's fit the model with the best parameters to the data with KFold.

In [ ]:
def fit_model(tuner, best_hps, X_train, y_train, X_test, features=features, n_fold=NFOLD, seed=SEED):
    cv = KFold(n_splits=n_fold, shuffle=True, random_state=seed)

    models = []
    oof_train = np.zeros((len(X_train),))
    y_preds = np.zeros((len(X_test),))

    for fold_id, (train_index, valid_index) in tqdm(enumerate(cv.split(X_train, y_train))):
        # split
        X_tr = X_train.loc[train_index, features].values
        X_val = X_train.loc[valid_index, features].values
        y_tr = y_train.loc[train_index].values
        y_val = y_train.loc[valid_index].values
        
        # model
        tf.keras.backend.clear_session()
        model = tuner.hypermodel.build(best_hps)
            
        # callbacks
        er = tf.keras.callbacks.EarlyStopping(patience=16, restore_best_weights=True, monitor='val_loss')
        ReduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, mode='min')
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f'mybestweight{fold_id}.hdf5', 
                                                              save_weights_only=True, verbose=0, monitor='val_loss', save_best_only=True)

        # fit
        history = model.fit(X_tr, y_tr, callbacks=[er, ReduceLR, model_checkpoint_callback], 
                            verbose=2, epochs=192, batch_size=best_hps.values['batch_size'],
                            validation_data=(X_val, y_val)) 
        
        # predict
        oof_train[valid_index] = model.predict(X_val).ravel()
        y_pred = model.predict(X_test[features].values).ravel()
        y_preds += y_pred / n_fold
        models.append(model)
        
    return oof_train, y_preds, models

In [ ]:
oof_train, y_preds, models = fit_model(tuner, best_hps, X_train, y_train, X_test, features=features, n_fold=NFOLD, seed=SEED)

# CV score

In [ ]:
print(f'CV (Tuned MLP): {mean_squared_error(y_train, oof_train, squared=False)}')

# Submit

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')
sub['target'] = y_preds
sub.to_csv('submission.csv', index=False)
sub.head()

All done, good job!